In [8]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
import plotly.graph_objects as go


In [9]:
reform = Reform.from_dict({
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

reform_taxable = Reform.from_dict({
  "gov.contrib.states.or.rebate.state_tax_exempt": {
    "2024-01-01.2100-12-31": True
  },
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

In [10]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
reformed_taxable = Microsimulation(reform=reform_taxable)


In [11]:
def calculate_poverty_impact_by_age(baseline, reformed, year):
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    age = baseline.calc("age", map_to="person", period=year)

    baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
    reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)

    results = {}
    age_groups = [
        (0, 18, "0-17"),
        (18, 65, "18-64"),
        (65, 200, "65+"),
        (0, 200, "Overall")
    ]

    for min_age, max_age, label in age_groups:
        if label == "Overall":
            mask = (state_codes == "OR")
        else:
            mask = (state_codes == "OR") & (age >= min_age) & (age < max_age)

        baseline_poverty_group = baseline_poverty[mask].mean()
        reform_poverty_group = reform_poverty[mask].mean()

        relative_poverty_reduction = (reform_poverty_group - baseline_poverty_group) / baseline_poverty_group
        results[label] = relative_poverty_reduction

    return results


In [12]:
# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame(columns=["year", "age_group", "relative_poverty_reduction", "relative_poverty_reduction_taxable"])


In [13]:
# Calculate for each year and add to the DataFrame
for year in range(2025, 2028):
    results_untaxed = calculate_poverty_impact_by_age(baseline, reformed, year)
    results_taxed = calculate_poverty_impact_by_age(baseline, reformed_taxable, year)

    for age_group in results_untaxed.keys():
        new_row = pd.DataFrame({
            "year": [year],
            "age_group": [age_group],
            "relative_poverty_reduction": [results_untaxed[age_group]],
            "relative_poverty_reduction_taxable": [results_taxed[age_group]]
        })
        all_results = pd.concat([all_results, new_row], ignore_index=True)


In [14]:
# Display the results
print(all_results)

    year age_group  relative_poverty_reduction  \
0   2025      0-17                   -0.392697   
1   2025     18-64                   -0.207817   
2   2025       65+                   -0.053603   
3   2025   Overall                   -0.216189   
4   2026      0-17                   -0.434428   
5   2026     18-64                   -0.235611   
6   2026       65+                   -0.050389   
7   2026   Overall                   -0.239394   
8   2027      0-17                   -0.423603   
9   2027     18-64                   -0.233850   
10  2027       65+                   -0.076262   
11  2027   Overall                   -0.241171   

    relative_poverty_reduction_taxable  
0                            -0.392697  
1                            -0.217906  
2                            -0.053603  
3                            -0.222731  
4                            -0.461682  
5                            -0.243466  
6                            -0.050389  
7                    

In [16]:
# Define colors for age groups
colors = {
    "0-17": {"untaxed": "#003366", "taxed": "#333333"},     # Very dark blue / Dark grey
    "18-64": {"untaxed": "#0052a3", "taxed": "#666666"},    # Dark blue / Medium grey
    "65+": {"untaxed": "#0066cc", "taxed": "#999999"},      # Medium blue / Light grey
    "Overall": {"untaxed": "#3399ff", "taxed": "#cccccc"},  # Light blue / Very light grey

}


# Create the plot
fig = go.Figure()

# Add traces for each age group
for age_group in colors.keys():
    data = all_results[all_results['age_group'] == age_group]

    # Untaxed scenario
    fig.add_trace(go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction'],
        mode='lines',
        name=f'{age_group} (Untaxed)',
        line=dict(color=colors[age_group]['untaxed'], width=2, dash='dot')
    ))

    # Taxed scenario
    fig.add_trace(go.Scatter(
        x=data['year'], 
        y=data['relative_poverty_reduction_taxable'],
        mode='lines',
        name=f'{age_group} (Taxed)',
        line=dict(color=colors[age_group]['taxed'], width=2, dash='solid')
    ))

# Update layout
fig.update_layout(
    title='Oregon Rebate Impact on Poverty by Age Group Over Time',
    xaxis_title='Year',
    yaxis_title='Relative Poverty Reduction (%)',
    legend_title='Age Group and Tax Status',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    ),
    height=600,
    width=800,
)

# Update x-axis to show only three steps
min_year = all_results['year'].min()
max_year = all_results['year'].max()
mid_year = min_year + (max_year - min_year) // 2

fig.update_xaxes(
    tickvals=[min_year, mid_year, max_year],
    ticktext=[str(min_year), str(mid_year), str(max_year)],
    tickformat='d'
)

# Update y-axis
fig.update_yaxes(tickformat='.0%')

# Update hover template
fig.update_traces(
    hovertemplate='Year: %{x}<br>Age Group: %{data.name}<br>Relative Poverty Reduction: %{y:.2%}<extra></extra>'
)

fig = format_fig(fig)
fig.show()